In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('WVL20-21data.txt')
winter2 = pd.read_csv ('WVL21-22data.txt')
winter3 = pd.read_csv ('WVL22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
precip_days3[676:690]

,station,datetime,precip(mm),wxcodes
1821,WVL,2023-01-23 15:31:00,0.0001,SN FZFG
1822,WVL,2023-01-23 15:39:00,0.0001,SN FZFG
1823,WVL,2023-01-23 15:42:00,0.0001,SN FZFG
1824,WVL,2023-01-23 15:51:00,0.0001,-SN FZFG
1825,WVL,2023-01-23 15:56:00,0.0001,SN FZFG
1826,WVL,2023-01-23 15:58:00,0.0001,SN FZFG
1827,WVL,2023-01-23 16:25:00,0.0001,-SN BR
1828,WVL,2023-01-23 16:56:00,0.0001,SN FZFG
1829,WVL,2023-01-23 17:03:00,0.0001,SN FZFG
1830,WVL,2023-01-23 17:23:00,0.0001,+SN FZFG


In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
wvl = LocationInfo(
    'Waterville', 'Maine', 
    latitude= 44.5520,
    longitude= -69.6317,
    timezone='America/Connecticut'
)

wvl

LocationInfo(name='Waterville', region='Maine', timezone='America/Connecticut', latitude=44.552, longitude=-69.6317)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(wvl.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(wvl.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(wvl.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:55:52  2020-12-01 15:59:28  2020-12-01
1   2020-12-02 06:56:58  2020-12-02 15:59:09  2020-12-02
2   2020-12-03 06:58:03  2020-12-03 15:58:52  2020-12-03
3   2020-12-04 06:59:07  2020-12-04 15:58:38  2020-12-04
4   2020-12-05 07:00:09  2020-12-05 15:58:27  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:24:11  2021-02-24 17:19:48  2021-02-24
86  2021-02-25 06:22:31  2021-02-25 17:21:09  2021-02-25
87  2021-02-26 06:20:50  2021-02-26 17:22:30  2021-02-26
88  2021-02-27 06:19:09  2021-02-27 17:23:50  2021-02-27
89  2021-02-28 06:17:26  2021-02-28 17:25:10  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:55:35  2021-12-01 15:59:33  2021-12-01
1   2021-12-02 06:56:42  2021-12-02 15:59:13  2021-12-02
2   2021-12-03 06:57:48  2021-12-03 15:58:56  2021-12-03
3   2021-12-04 06:58:52  2021-12-04 15:58:41  2021-12-04
4   2021-12-05 06:59:54  2021-12-05 15:58:29  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:24:35  2022-02-24 17:19:28  2022-02-24
86  2022-02-25 06:22:56  2022-02-25 17:20:49  2022-02-25
87  2022-02-26 06:21:15  2022-02-26 17:22:10  2022-02-26
88  2022-02-27 06:19:33  2022-02-27 17:23:31  2022-02-27
89  2022-02-28 06:17:51  2022-02-28 17:24:51  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:19  2022-12-01 15:59:37  2022-12-01
1   2022-12-02 06:56:26  2022-12-02 15:59:17  2022-12-02
2   2022-12-03 06:57:32  2022-12-03 15:58:59  2022-12-03
3   2022-12-04 06:58:36  2022-12-04 15:58:44  2022-12-04
4   2022-12-05 06:59:39  2022-12-05 15:58:31  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:24:59  2023-02-24 17:19:08  2023-02-24
86  2023-02-25 06:23:20  2023-02-25 17:20:29  2023-02-25
87  2023-02-26 06:21:39  2023-02-26 17:21:50  2023-02-26
88  2023-02-27 06:19:58  2023-02-27 17:23:11  2023-02-27
89  2023-02-28 06:18:16  2023-02-28 17:24:31  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,WVL,2020-12-01 00:56:00,2.7900,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
1,WVL,2020-12-01 01:56:00,2.5400,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
2,WVL,2020-12-01 02:56:00,2.0300,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
3,WVL,2020-12-01 03:56:00,2.0300,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
4,WVL,2020-12-01 04:56:00,1.7800,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
...,...,...,...,...,...,...,...
351,WVL,2021-02-27 18:56:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50
352,WVL,2021-02-27 19:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50
353,WVL,2021-02-27 20:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50
354,WVL,2021-02-27 21:56:00,0.0000,-RA BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,WVL,2021-12-02 11:56:00,0.0001,-SN,2021-12-02,2021-12-02 06:56:42,2021-12-02 15:59:13
1,WVL,2021-12-02 12:56:00,0.0001,-SN BR,2021-12-02,2021-12-02 06:56:42,2021-12-02 15:59:13
2,WVL,2021-12-02 13:56:00,0.2500,-SN,2021-12-02,2021-12-02 06:56:42,2021-12-02 15:59:13
3,WVL,2021-12-02 14:56:00,0.5100,UP,2021-12-02,2021-12-02 06:56:42,2021-12-02 15:59:13
4,WVL,2021-12-02 15:56:00,0.5100,UP BR,2021-12-02,2021-12-02 06:56:42,2021-12-02 15:59:13
...,...,...,...,...,...,...,...
333,WVL,2022-02-25 22:56:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:22:56,2022-02-25 17:20:49
334,WVL,2022-02-25 23:56:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:22:56,2022-02-25 17:20:49
335,WVL,2022-02-26 00:56:00,0.0001,-SN,2022-02-26,2022-02-26 06:21:15,2022-02-26 17:22:10
336,WVL,2022-02-27 19:56:00,0.0001,-RA,2022-02-27,2022-02-27 06:19:33,2022-02-27 17:23:31


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,WVL,2022-12-01 00:56:00,5.0800,RA BR,2022-12-01,2022-12-01 06:55:19,2022-12-01 15:59:37
1,WVL,2022-12-01 01:56:00,6.3500,+RA BR,2022-12-01,2022-12-01 06:55:19,2022-12-01 15:59:37
2,WVL,2022-12-01 02:56:00,2.5400,-RA,2022-12-01,2022-12-01 06:55:19,2022-12-01 15:59:37
3,WVL,2022-12-01 03:56:00,0.2500,RA,2022-12-01,2022-12-01 06:55:19,2022-12-01 15:59:37
4,WVL,2022-12-01 17:56:00,0.0001,-RA,2022-12-01,2022-12-01 06:55:19,2022-12-01 15:59:37
...,...,...,...,...,...,...,...
391,WVL,2023-02-26 23:56:00,0.0001,-SN,2023-02-26,2023-02-26 06:21:39,2023-02-26 17:21:50
392,WVL,2023-02-27 01:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:19:58,2023-02-27 17:23:11
393,WVL,2023-02-27 03:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:19:58,2023-02-27 17:23:11
394,WVL,2023-02-27 04:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:19:58,2023-02-27 17:23:11


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
6,WVL,2020-12-01 06:56:00,3.0500,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
7,WVL,2020-12-01 07:56:00,0.7600,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
8,WVL,2020-12-01 08:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
9,WVL,2020-12-01 09:56:00,1.0200,+RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
10,WVL,2020-12-01 10:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
...,...,...,...,...,...,...,...
334,WVL,2021-02-22 16:56:00,0.0001,-SN,2021-02-22,2021-02-22 06:27:29,2021-02-22 17:17:05
346,WVL,2021-02-24 06:56:00,0.0001,UP,2021-02-24,2021-02-24 06:24:11,2021-02-24 17:19:48
347,WVL,2021-02-25 08:56:00,0.0001,-RA,2021-02-25,2021-02-25 06:22:31,2021-02-25 17:21:09
348,WVL,2021-02-27 15:56:00,0.0001,UP,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
7,WVL,2020-12-01 07:56:00,0.7600,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28,2020-12-01 07:55:52,2020-12-01 14:59:28
8,WVL,2020-12-01 08:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28,2020-12-01 07:55:52,2020-12-01 14:59:28
9,WVL,2020-12-01 09:56:00,1.0200,+RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28,2020-12-01 07:55:52,2020-12-01 14:59:28
10,WVL,2020-12-01 10:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28,2020-12-01 07:55:52,2020-12-01 14:59:28
11,WVL,2020-12-01 11:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28,2020-12-01 07:55:52,2020-12-01 14:59:28
...,...,...,...,...,...,...,...,...,...
320,WVL,2021-02-16 13:56:00,0.2500,-SN BR,2021-02-16,2021-02-16 06:36:57,2021-02-16 17:08:49,2021-02-16 07:36:57,2021-02-16 16:08:49
321,WVL,2021-02-16 14:56:00,0.0001,-SN BR,2021-02-16,2021-02-16 06:36:57,2021-02-16 17:08:49,2021-02-16 07:36:57,2021-02-16 16:08:49
322,WVL,2021-02-16 15:56:00,0.5100,-SN BR,2021-02-16,2021-02-16 06:36:57,2021-02-16 17:08:49,2021-02-16 07:36:57,2021-02-16 16:08:49
347,WVL,2021-02-25 08:56:00,0.0001,-RA,2021-02-25,2021-02-25 06:22:31,2021-02-25 17:21:09,2021-02-25 07:22:31,2021-02-25 16:21:09


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.13742110091743118
2021-2022: 0.15654607843137255
2022-2023: 0.5561513043478261


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,WVL,2020-12-01 00:56:00,2.7900,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
1,WVL,2020-12-01 01:56:00,2.5400,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
2,WVL,2020-12-01 02:56:00,2.0300,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
3,WVL,2020-12-01 03:56:00,2.0300,RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
4,WVL,2020-12-01 04:56:00,1.7800,-RA BR,2020-12-01,2020-12-01 06:55:52,2020-12-01 15:59:28
...,...,...,...,...,...,...,...
351,WVL,2021-02-27 18:56:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50
352,WVL,2021-02-27 19:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50
353,WVL,2021-02-27 20:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50
354,WVL,2021-02-27 21:56:00,0.0000,-RA BR,2021-02-27,2021-02-27 06:19:09,2021-02-27 17:23:50


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19488\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.23154000000000008
2021-2022: 0.24340404040404037
2022-2023: 0.4613721739130435


In [68]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'ME'

data.to_csv('wvl.csv', index=False)